In [32]:
import numpy as np
from pymoo.core.problem import Problem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.sampling.rnd import FloatRandomSampling
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PolynomialMutation
from pymoo.termination import get_termination
from pymoo.optimize import minimize


TIME_FRAMES = [
    (1, "10:00 PM to 6:00 AM"),
    (2, "6:00 AM to 6:00 PM"),
    (3, "6:00 PM to 10:00 PM"),
]


grid_tariffs = [
    {"units": 40, "price": 1.50},
    {"units": 10, "price": 3.25},
    {"units": 50, "price": 4.05},
    {"units": 50, "price": 5.10},
    {"units": 50, "price": 6.95},
    {"units": 50, "price": 8.20}
]

sellers = [
    {"id": 1, "time_slot": 1, "node_id": 1, "max_generation": 30, "price": 2.8},
    {"id": 2, "time_slot": 1, "node_id": 2, "max_generation": 25, "price": 3.0},
    {"id": 3, "time_slot": 1, "node_id": 3, "max_generation": 35, "price": 4.0},
    {"id": 4, "time_slot": 1, "node_id": 3, "max_generation": 28, "price": 3.5},
    
    {"id": 5, "time_slot": 2, "node_id": 1, "max_generation": 60, "price": 5.2},
    {"id": 6, "time_slot": 2, "node_id": 2, "max_generation": 55, "price": 6.0},
    {"id": 7, "time_slot": 2, "node_id": 3, "max_generation": 70, "price": 5.5},
    {"id": 8, "time_slot": 2, "node_id": 1, "max_generation": 80, "price": 5.3},
    {"id": 9, "time_slot": 2, "node_id": 2, "max_generation": 75, "price": 5.8},
    {"id": 10, "time_slot": 2, "node_id": 3, "max_generation": 65, "price": 5.4},

    {"id": 9, "time_slot": 3, "node_id": 2, "max_generation": 80, "price": 6.8},
    {"id": 10, "time_slot": 3, "node_id": 1, "max_generation": 90, "price": 6.9},
    {"id": 11, "time_slot": 3, "node_id": 3, "max_generation": 95, "price": 7.0},
]

buyers = [
    {"id": 1, "time_slot": 1, "node_id": 1, "demand": 20, "price": 4.0},
    {"id": 2, "time_slot": 1, "node_id": 2, "demand": 15, "price": 4.5},
    {"id": 3, "time_slot": 1, "node_id": 3, "demand": 10, "price": 3.8},
    
    {"id": 4, "time_slot": 2, "node_id": 1, "demand": 50, "price": 6.5},
    {"id": 5, "time_slot": 2, "node_id": 2, "demand": 40, "price": 6.0},
    {"id": 6, "time_slot": 2, "node_id": 3, "demand": 35, "price": 7.0},
    {"id": 7, "time_slot": 2, "node_id": 1, "demand": 45, "price": 6.2},
    
    {"id": 8, "time_slot": 3, "node_id": 2, "demand": 60, "price": 7.5},
    {"id": 9, "time_slot": 3, "node_id": 3, "demand": 70, "price": 7.8},
    {"id": 10, "time_slot": 3, "node_id": 1, "demand": 75, "price": 7.0}
]




def get_grid_price(buyers, grid_tariffs):
    total_load_demand = sum(buyer["demand"] for buyer in buyers)
    cumulative_units = 0
    for slab in grid_tariffs:
        cumulative_units += slab["units"]
        if total_load_demand <= cumulative_units:
            return slab["price"]
    raise ValueError("Total load demand exceeds all defined tariff slabs.")


def calculate_mcp(sellers, buyers, grid_tariffs):
    offers = sorted(sellers, key=lambda x: x["price"])
    total_demand = sum(buyer["demand"] for buyer in buyers)
    buyer_grid_price = get_grid_price(buyers, grid_tariffs)
    cumulative_supply = 0
    mcp = None
    for offer in offers:
        cumulative_supply += offer["max_generation"]
        if cumulative_supply >= total_demand:
            mcp = min(offer["price"], buyer_grid_price - 0.5)
            break
    if mcp is not None:
        return mcp
    raise ValueError("Insufficient supply to meet demand")


class EnergyTradingProblem(Problem):
    def __init__(self, sellers, buyers, mcp):
        self.sellers = sellers
        self.buyers = buyers
        self.mcp = mcp

        xl = np.zeros(len(sellers) * len(buyers))
        xu = np.array([min(seller["max_generation"], buyer["demand"]) for seller in sellers for buyer in buyers])

        super().__init__(n_var=len(sellers) * len(buyers), n_obj=2, n_constr=len(sellers) + len(buyers), xl=xl, xu=xu)

    def _evaluate(self, X, out, *args, **kwargs):
        n_sellers = len(self.sellers)
        n_buyers = len(self.buyers)

        transactions = X.reshape((-1, n_sellers, n_buyers))
        welfare_sellers = np.zeros(transactions.shape[0])
        welfare_buyers = np.zeros(transactions.shape[0])
        g = np.zeros((transactions.shape[0], n_sellers + n_buyers))

        for i in range(transactions.shape[0]):
            total_seller_welfare = 0
            total_buyer_welfare = 0

            for j, seller in enumerate(self.sellers):
                transacted_energy = np.sum(transactions[i, j, :])
                total_seller_welfare += transacted_energy * self.mcp
                g[i, j] = transacted_energy - seller["max_generation"]

            for k, buyer in enumerate(self.buyers):
                transacted_energy = np.sum(transactions[i, :, k])
                if transacted_energy > 0:
                    total_buyer_welfare += buyer["price"] * np.log(transacted_energy + 1e-6) - self.mcp * transacted_energy
                g[i, n_sellers + k] = transacted_energy - buyer["demand"]

            welfare_sellers[i] = total_seller_welfare
            welfare_buyers[i] = total_buyer_welfare

        out["F"] = np.column_stack([-welfare_sellers, -welfare_buyers])
        out["G"] = g


def run_bfs(data, voltage_limits=(0.95, 1.05)):
    node_voltages = {node["node_id"]: 1.0 for node in data["sellers"] + data["buyers"]}
    power_injection = {node_id: 0.0 for node_id in node_voltages.keys()}
    
    for seller, row in zip(data["sellers"], data["transactions"]):
        node_id = seller["node_id"]
        transacted_energy = sum(row)
        power_injection[node_id] += transacted_energy

    for buyer_idx, buyer in enumerate(data["buyers"]):
        node_id = buyer["node_id"]
        transacted_energy = sum(data["transactions"][:, buyer_idx])
        power_injection[node_id] -= transacted_energy

    current = {node_id: 0.0 for node_id in node_voltages.keys()}
    for node_id in reversed(sorted(node_voltages.keys())):
        if node_id in power_injection:
            current[node_id] = power_injection[node_id] / node_voltages[node_id]

    for node_id in sorted(node_voltages.keys()):
        line_impedance = 0.01
        if node_id > 1:
            prev_node_id = node_id - 1
            voltage_drop = current[node_id] * line_impedance
            node_voltages[node_id] = node_voltages[prev_node_id] - voltage_drop

    min_voltage, max_voltage = voltage_limits
    voltage_violations = [
        node_id for node_id, voltage in node_voltages.items()
        if voltage < min_voltage or voltage > max_voltage
    ]

    return voltage_violations



for time_frame, description in TIME_FRAMES:
    print(f"\nTime Frame {time_frame}: {description}")
    filtered_sellers = [s for s in sellers if s["time_slot"] == time_frame]
    filtered_buyers = [b for b in buyers if b["time_slot"] == time_frame]

    if not filtered_sellers or not filtered_buyers:
        print(f"No participants for Time Frame {time_frame}. Skipping.")
        continue

    adjusted_sellers = filtered_sellers.copy()
    iteration = 0

    while True:
        iteration += 1
        print(f"Iteration: {iteration}")

        try:
            mcp = calculate_mcp(adjusted_sellers, filtered_buyers, grid_tariffs)
            print(f"Market Clearing Price (MCP): {mcp}")
        except Exception as e:
            print(f"Error calculating MCP: {e}")
            break

        problem = EnergyTradingProblem(adjusted_sellers, filtered_buyers, mcp)

        algorithm = NSGA2(
            pop_size=100,
            sampling=FloatRandomSampling(),
            crossover=SBX(prob=0.9, eta=15),
            mutation=PolynomialMutation(prob=0.1, eta=20),
            eliminate_duplicates=True,
        )

        termination = get_termination("n_gen", 200)

        result = minimize(problem, algorithm, termination, seed=1, verbose=False)

        pareto_front = result.F
        best_idx = np.argmin(np.sum(pareto_front, axis=1))
        best_solution = result.X[best_idx]
        best_solution_reshaped = best_solution.reshape(len(adjusted_sellers), len(filtered_buyers))

        print(f"Best Transactions:\n{best_solution_reshaped}")
        print("-" * 50)

        violated_nodes = run_bfs({"transactions": best_solution_reshaped, "sellers": adjusted_sellers, "buyers": filtered_buyers})

        if not violated_nodes:
            print("Feasible solution found for this time frame.")
            break

        print(f"Voltage violations at nodes: {violated_nodes}")

        for seller in adjusted_sellers:
            if seller["node_id"] in violated_nodes:
                seller["max_generation"] *= 0.9

    print(f"Completed optimization for Time Frame {time_frame}.")





Time Frame 1: 10:00 PM to 6:00 AM
Iteration: 1
Market Clearing Price (MCP): 2.75
Best Transactions:
[[2.71949808 6.53962238 0.05489818]
 [6.11195621 1.01551158 2.31286879]
 [2.25177578 6.81025163 4.32388108]
 [8.83489856 0.61937035 3.28036651]]
--------------------------------------------------
Voltage violations at nodes: [2, 3]
Iteration: 2
Market Clearing Price (MCP): 2.75
Best Transactions:
[[3.18673691 1.10456104 4.04278722]
 [9.97420158 6.80432373 1.29180162]
 [0.95572706 3.35857218 4.01673018]
 [5.87527046 3.72750893 0.61945879]]
--------------------------------------------------
Voltage violations at nodes: [3]
Iteration: 3
Market Clearing Price (MCP): 2.75
Best Transactions:
[[8.52531762 5.96179069 0.19063245]
 [9.55172787 0.87880483 4.59133608]
 [1.64378968 7.64038651 4.25366224]
 [0.24458701 0.4846606  0.94948268]]
--------------------------------------------------
Voltage violations at nodes: [3]
Iteration: 4
Market Clearing Price (MCP): 2.75
Best Transactions:
[[ 5.376706